### 1

hisat2-build -p 6 ref.fa ref

hisat2 -p 6 -x ref -1 SRR453566_1.fastq -2 SRR453566_2.fastq -S aligned.sam

### 2

3554 genes expressed at level greater than median across dataset (TPM>36)

6648 genes expressed at level TPM>5

### 3

rnaspades.py -1 SRR453566_1.fastq -2 SRR453566_2.fastq -t 6 -m 15 -o asm

rnaQUAST.py -r ref.fa --gtf genes.gtf -c asm/transcripts.fasta -t 6 -o asm_quast

#### 95%-assembled genes: 3724

#### 95%-covered genes: 3831

### Выводы:

Выше среднего экспрессируется примерно столько же генов, сколько получается покрыто при сборке.

При этом выравнивание генов на референс удобнее использовать при сравнительном анализе, так как не надо выбирать абсолютные отсечки TPM (FPKM), а только сравнивать относительные уровни экспрессии в разных образцах.

In [3]:
import pysam
from Bio import SeqIO
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import gffutils

In [6]:
samfile = pysam.AlignmentFile("aligned_sorted.bam", "rb")
genome = next(SeqIO.parse("ref.fa", "fasta")).seq
#db = gffutils.create_db("genes.gtf", dbfn='genes.db', merge_strategy="create_unique")
db = gffutils.FeatureDB('genes.db')

In [10]:

for f in db.features_of_type(featuretype="gene"):
    help(f)
    break

Help on Feature in module gffutils.feature object:

class Feature(builtins.object)
 |  Feature(seqid='.', source='.', featuretype='.', start='.', end='.', score='.', strand='.', frame='.', attributes=None, extra=None, bin=None, id=None, dialect=None, file_order=None, keep_order=False, sort_attribute_values=False)
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __getitem__(self, key)
 |  
 |  __hash__(self)
 |      Return hash(self).
 |  
 |  __init__(self, seqid='.', source='.', featuretype='.', start='.', end='.', score='.', strand='.', frame='.', attributes=None, extra=None, bin=None, id=None, dialect=None, file_order=None, keep_order=False, sort_attribute_values=False)
 |      Represents a feature from the database.
 |      
 |      Usually you won't want to use this directly, since it has various
 |      implementation details needed for operating in the context of FeatureDB
 |      objects.  Instead, try the :func:`gffutils.feature.

In [22]:
rpks = []
covs = []

for f in db.features_of_type(featuretype="gene"):
    id = f.seqid
    start = f.start
    end = f.end
    l = end - start
    cov = [0] * l
    reads = set()
    for x in samfile.pileup(id, start=start-1, end=end-1, truncate=True):
        reads.update(x.get_query_names())
        cov[x.pos-start] = x.get_num_aligned()
    #print(start, end, l, len(cov))
    covs.append(np.count_nonzero(cov)/len(cov))
    rpks.append(len(reads) / l * 1000)


print(rpks)
print(covs)

[289.3136403127715, 202.7027027027027, 33.898305084745765, 61.50978564771668, 16.441573693482088, 8.806262230919765, 100.10111223458038, 62.32294617563739, 175.2021563342318, 14.598540145985401, 88.11005568293481, 51.85497470489039, 47.65146358066713, 295.22184300341297, 658.8486140724946, 743.5387673956262, 132.14285714285714, 88.66279069767442, 2390.909090909091, 147.46816701214095, 147.96425024826215, 6715.9152634437805, 7998.439937597504, 363.6363636363636, 594.5179584120983, 282.1752265861027, 191.3801949717804, 53.82165605095542, 48.43918191603875, 74.41016333938293, 51.18961788031723, 483.7476099426386, 36.72612801678909, 516.8539325842697, 81.69440242057487, 137.888956680903, 17.627441638875656, 676.5140324963072, 1786.392405063291, 84.07871198568873, 0.0, 19.24198250728863, 110.89303238469088, 2474.468085106383, 210.3960396039604, 76.48601398601399, 495.42217700915563, 71.12616426756986, 150.18706574024586, 634.1789052069427, 265.1162790697674, 178.4452296819788, 26.1754726126

In [64]:
total = sum(rpks)
tpms = [i*1000000/total for i in rpks]

In [67]:
good = 0
for a, b in zip(covs, tpms):
    if a > 0.95 and b > np.median(tpms):
        good += 1
print(good, f"genes expressed at level greater than median across dataset (TPM>{round(np.median(tpms))})")

good = 0
for a, b in zip(covs, tpms):
    if a > 0.95 and b > 5:
        good += 1
print(good, "genes expressed at level TPM>5")

3554 genes expressed at level greater than median across dataset (TPM>36)
6648 genes expressed at level TPM>5


In [68]:
import matplotlib.pyplot as plt
from scipy import stats

print(stats.describe(tpms))
print(np.percentile(tpms, range(0, 101, 5)))
print(np.median(tpms))

DescribeResult(nobs=7126, minmax=(0.0, 14324.144080758128), mean=140.3311815885493, variance=245698.57881147997, skewness=12.276874842238056, kurtosis=239.52753202519852)
[0.00000000e+00 3.88185042e+00 7.98434591e+00 1.16021856e+01
 1.45774065e+01 1.75650965e+01 2.06601237e+01 2.36098575e+01
 2.71844346e+01 3.08583660e+01 3.56902171e+01 4.10673224e+01
 4.77948289e+01 5.62443447e+01 6.78738705e+01 8.40751776e+01
 1.07715519e+02 1.51235157e+02 2.34526252e+02 5.22741372e+02
 1.43241441e+04]
35.690217127804075
